In [1]:
import os
from tqdm import tqdm
import numpy as np
from imgaug import augmenters as iaa
from PIL import Image

In [2]:
PATH = 'dataset'

In [3]:
def augment_images_in_folder(folder_path):
    # Создаем объект аугментации
    seq = iaa.Sequential([
        iaa.Fliplr(0.7),
        iaa.Flipud(0.7),
        iaa.Affine(rotate=(-180, 180)),
        iaa.GaussianBlur(sigma=(0, 1.0)),
        iaa.AdditiveGaussianNoise(scale=(0, 0.1 * 255))
    ])

    image_files = os.listdir(folder_path)

    bar = tqdm(enumerate(image_files), total=len(image_files), ncols=100)
    bar.set_description(f"Augmentation: ")
                    
    for i, file_name in bar:
        if not file_name.startswith('.'):
            image_path = os.path.join(folder_path, file_name)
            try:
                image = Image.open(image_path)

                if image.mode == 'RGBA':
                    image = image.convert('RGB')

                image_array = np.array(image)
                images_aug = [seq.augment_image(image_array) for _ in range(50)]
                    
                for i, image_aug in enumerate(images_aug):
                    class_name = os.path.basename(folder_path)
                    new_folder = os.path.join(PATH, f"{class_name}_aug")
                    os.makedirs(new_folder, exist_ok=True)
                    new_file_name = f"{os.path.splitext(file_name)[0]}_aug_{i}.jpg"
                    Image.fromarray(image_aug).save(os.path.join(new_folder, new_file_name))
            except Exception as e:
                print(f"Ошибка при обработке изображения {file_name}: {e}")


In [4]:
class_folders = [f.path for f in os.scandir(PATH) if f.is_dir()]

In [5]:
for folder in class_folders:
    augment_images_in_folder(folder)

Augmentation: : 100%|█████████████████████████████████████████████| 308/308 [00:52<00:00,  5.81it/s]
